In [1]:
!pip install ultralytics opencv-python-headless matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.7/40.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.9/777.9 kB 13.5 MB/s eta 0:00:00a 0:00:01


In [4]:
import os
import glob
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
from ultralytics import YOLO
import wandb
from kaggle_secrets import UserSecretsClient

In [5]:
# WandB API anahtarınızı girin
user_secrets = UserSecretsClient()
wandb_api_key = user_secrets.get_secret("WANDB_API_KEY")
wandb.login(key=wandb_api_key)

data_dir = '/kaggle/input/militaryaircraftdetectiondataset/dataset'
output_dir = '/kaggle/working/'

merged_csv = os.path.join(output_dir, 'merged_annotations.csv')

all_files = glob.glob(os.path.join(data_dir, "*.csv"))

if not all_files:
    raise FileNotFoundError(f"No CSV files found in directory: {data_dir}")

df_list = []
for file in all_files:
    try:
        df = pd.read_csv(file)
        df_list.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    merged_df.to_csv(merged_csv, index=False)
else:
    raise ValueError("No dataframes to concatenate")


wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [6]:
# Sınıf isimlerini ve id'lerini belirleyin
class_names = ['MQ9', 'RQ4']  # Bu isimler verisetinden seçilecek aircraft modeline göre değiştirilmeli
num_classes = len(class_names)
class_id_map = {name: idx for idx, name in enumerate(class_names)}

# YOLO formatına dönüşüm
def convert_to_yolo_format(row):
    dw = 1. / row['width']
    dh = 1. / row['height']
    x_center = (row['xmin'] + row['xmax']) / 2.0 * dw
    y_center = (row['ymin'] + row['ymax']) / 2.0 * dh
    width = (row['xmax'] - row['xmin']) * dw
    height = (row['ymax'] - row['ymin']) * dh
    return [class_id_map[row['class']], x_center, y_center, width, height]


train_df, val_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# Verilerdeki sınıf etiketlerini kontrol edin ve temizleyin
def validate_and_clean_labels(df):
    valid_rows = []
    for _, row in df.iterrows():
        if row['class'] in class_id_map:
            valid_rows.append(row)
        else:
            print(f"Invalid class {row['class']} in {row['filename']}")
    return pd.DataFrame(valid_rows)

train_df = validate_and_clean_labels(train_df)
val_df = validate_and_clean_labels(val_df)


Invalid class Mig31 in a2b0ee67bdcae1730ea4701b35f871e3
Invalid class F15 in 5dc92840a1c2570658f210118ee712af
Invalid class Mirage2000 in a2d982a9b72617151f260480fd46fe9e
Invalid class F16 in 4566d3a226f58658aafa34b6a03b75d4
Invalid class KC135 in a56e2ec4e5f00ab4599bd8d9030dbed2
Invalid class F16 in 902e39cfdebc228f7ce58f77dec6f596
Invalid class EF2000 in c4fa079cdf2d1d4be6b3a460f9b44477
Invalid class A10 in 43bb9d080bad94c5149bb277164e61a8
Invalid class F22 in 82c77d929dfd3df9203e93f40608d6df
Invalid class B2 in 4490e4648ccdf9232fd6ba9319f54bfd
Invalid class Mirage2000 in b0aa6f15e9602b69a098c34a46443142
Invalid class Su57 in dd5e7c97da847ca79ba029a57586925a
Invalid class F22 in 5ecfc397468fcf2ad8bc506b038c3760
Invalid class Mirage2000 in b9ee4d8b503fe02318cd302ca99503b7
Invalid class F18 in 7ed485d1a45ed03b6b16474d71f0893b
Invalid class F15 in bb7eb8acb1fbf0863ca958bc83dc7b80
Invalid class H6 in 51110e7b381f1c73288b8bf699fd59af
Invalid class J20 in 9882b204f2ebec7a94faa590d3cd0117
I

In [ ]:
# YOLO formatında dosyaları oluşturun
def create_yolo_files(df, split):
    for _, row in df.iterrows():
        yolo_data = convert_to_yolo_format(row)
        txt_filename = os.path.join(output_dir, split, os.path.splitext(row['filename'])[0] + '.txt')
        img_filename = os.path.join(data_dir, row['filename'] + '.jpg')  # Resim dosyasının tam yolu
        output_img_filename = os.path.join(output_dir, split, row['filename'] + '.jpg')
        
        os.makedirs(os.path.dirname(txt_filename), exist_ok=True)
        
        with open(txt_filename, 'w') as f:
            f.write(' '.join(map(str, yolo_data)) + '\n')
        
        # Resmi kopyala
        if os.path.exists(img_filename):
            shutil.copy(img_filename, output_img_filename)
        else:
            print(f"Warning: {img_filename} not found!")


In [ ]:
# Train ve val dosyalarını ve resimlerini oluşturun
create_yolo_files(train_df, 'train')
create_yolo_files(val_df, 'val')

# YOLOv8 modelini tanımlayın
model = YOLO('yolov8n.pt')  # YOLOv8 nano model, ihtiyaca göre değiştirilebilir

# Eğitim verisetinin ve doğrulama verisetinin yollarını ayarlayın
data_yaml = """
train: /kaggle/working/train
val: /kaggle/working/val
nc: {num_classes}
names: {class_names}
"""

# data.yaml dosyasını oluşturun
data_yaml = data_yaml.format(num_classes=num_classes, class_names=class_names)
data_yaml_path = '/kaggle/working/data.yaml'
with open(data_yaml_path, 'w') as f:
    f.write(data_yaml)


In [ ]:
# Eğitim ayarlarını belirleyin
training_config = {
    'data': data_yaml_path,
    'epochs': 100,
    'batch': 16,
    'imgsz': 640,
    'device': '0,1',  # İki GPU'yu kullanmak için
    'project': 'AircraftDetectYolov8',
    'name': 'UAV_training',
    'exist_ok': True,
    'workers': 4,
    'lr0': 0.002,  # Başlangıç öğrenme hızı
}


In [ ]:
# CUDA hataları
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Modeli eğitin
model.train(**training_config)

# Modeli değerlendirin
metrics = model.val(data=data_yaml_path)
